In [2]:
#Import the relevant packages

import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.0'

In [ ]:
#Load the data
product_data = pd.read_csv('pyblp_v5.csv')
product_data.head()

In [ ]:
#First estimate with BLP:

#Formulate the variables for the linear parameters:
X1_formulation = pyblp.Formulation('1 + prices +  alcohol + whiskey + vodka + rum + american')

#Formulate the variables for the non-linear parameters:
X2_formulation = pyblp.Formulation('0 + prices')

#Obtain product formulation
product_formulations = (X1_formulation, X2_formulation)
product_formulations

In [ ]:
#Specify the method to approximate the integral in order to obtain mean utility
#, in our case we make use of Gauss-Hermite Quadrature Rule
pr_integration = pyblp.Integration('product', size=20)
pr_integration

In [ ]:
#Specify the optimization algorithm, we use BFGS
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})
bfgs

In [ ]:
#Formulate the problem
pr_problem = pyblp.Problem(product_formulations, product_data, integration=pr_integration)
pr_problem

In [ ]:
#Obtain results for BLP:
blp_results = pr_problem.solve(sigma=1, optimization=bfgs)
blp_results

In [ ]:
#Create function for RCNL:

def solve_rcnl(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['demand_instruments2'] = groups['shares'].transform(np.size)
    X1_formulation = pyblp.Formulation('1 + prices +  alcohol + american')
    X2_formulation = pyblp.Formulation('0 + prices')
    product_formulations = (X1_formulation, X2_formulation)
    pr_integration = pyblp.Integration('product', size=20)
    bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})
    pr_problem = pyblp.Problem(product_formulations, df, integration=pr_integration)
    results = pr_problem.solve(sigma=1, optimization=bfgs,rho=0.7)
    return results

In [ ]:
df2 = product_data.copy()
df2['nesting_ids'] = df2['whiskey']
rum = df2['rum'] == 1
other = df2['product_ids'].isin([13,19,14,20,15])
df2.loc[rum,'nesting_ids'] = 3
df2.loc[other,'nesting_ids'] = 4

In [ ]:
rcnl_results = solve_rcnl(df2)
rcnl_results